In [ ]:
%pip install erddapy

In [ ]:
import pandas as pd
from erddapy import ERDDAP

erddap_obj = ERDDAP(
    server='https://coastwatch.noaa.gov/erddap',
    protocol='griddap',
    response='nc'  # Download as NetCDF (compatible with Xarray)
)

Like the issue with AQS versus AirNow, NOAA CoastWatch data includes versions that feature near real-time data and versions that are delayed for quality control purposes.

We'll use a version that is quality controlled for scientific research here, but it is possible to access realtime data if you are interested in fetching a dataset that is related to a current event of interest by selecting a different DATASET_ID. Note that those data have not gone through the same rigor of validation processes. 

NOTE If you need data older than 2016, you should look for the "Reprocessed" or "Science Quality" dataset on the NOAA ERDDAP site, often labeled as noaacwSlaDaily or similar.

Visit https://coastwatch.noaa.gov/cw_html/cwViewer.html
For documentation about the API check out https://coastwatch.noaa.gov/erddap/index.html
A list of all datasets is here: https://coastwatch.noaa.gov/erddap/info/index.html?page=1&itemsPerPage=2000 (Note not all datasets are publicly accessible)

In [ ]:
# Recommended by Gemini; REVSIT THIS
# The dataset IDs are mapped to measures in last column of dataset list above
DATASET_ID = 'noaacwBLENDEDsshDaily' 

In [ ]:
LAT_MIN = 24.0
LAT_MAX = 30.0
LON_MIN = -90.0
LON_MAX = -80.0

In [ ]:
TARGET_DATE_STR = '2023-09-15' 

In [ ]:
from datetime import datetime, timedelta

target_date = datetime.strptime(TARGET_DATE_STR, '%Y-%m-%d')

window_start = target_date + timedelta(days=5)
window_end = target_date - timedelta(days=5)

In [ ]:
print(f"Fetching data from {DATASET_ID}...")
print(f"Time: {window_start} to {window_end}")
print(f"Box: Lat({LAT_MIN}), Lon({LON_MAX})")
    
erddap_obj.dataset_id = DATASET_ID
    
# Set constraints
erddap_obj.griddap_initialize()

erddap_obj.constraints['time>='] = window_start
erddap_obj.constraints['time<='] = window_end
erddap_obj.constraints['latitude>='] = LAT_MIN
erddap_obj.constraints['latitude<='] = LAT_MAX
erddap_obj.constraints['longitude>='] = LON_MIN
erddap_obj.constraints['longitude<='] = LON_MAX

Select the data to fetch below. "sla" is sea level anomaly (the deviation from mean sea surface hieght).

In [ ]:
# Select variables to download (ssh/sla and coordinates)
erddap_obj.variables = ['sla']

In [ ]:
data = erddap_obj.to_pandas()

In [ ]:
data

In [ ]:
data.to_csv('sealevel.csv')